In [3]:
import os, pathlib, shutil, random
from pathlib import Path
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization

In [9]:
#Global variables
percentVal=.2
batch_size = 32
maxTokens = 20000 # max voclbulary size
max_length = 600 # max number of words in document
epochCount=1
embed_dim = 256 # for Transformer implementation
num_heads = 2 # for Transformer implementation
dense_dim = 32 # for Transformer implementation
posString = "That was an excellent movie, I loved it."
negString = "That was a horrible movie, I hated it."
nlpRoot="C:/Users/lerma/machineLearning/NLP"
trainPath=nlpRoot+'/aclImdb/train'

In [5]:
#fetch files and untar, to be performed only once in project
# !curl -O https://ai.stanford.edu/~amaas/data/sentiment/aclImdb_v1.tar.gz
# !tar -xf aclImdb_v1.tar.gz

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed

  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0
  0     0    0     0    0     0      0      0 --:--:--  0:00:02 --:--:--     0
  0 80.2M    0  176k    0     0  51793      0  0:27:04  0:00:03  0:27:01 51803
  7 80.2M    7 6080k    0     0  1367k      0  0:01:00  0:00:04  0:00:56 1367k
 20 80.2M   20 16.6M    0     0  3133k      0  0:00:26  0:00:05  0:00:21 3133k
 34 80.2M   34 27.3M    0     0  4349k      0  0:00:18  0:00:06  0:00:12 4672k
 47 80.2M   47 38.0M    0     0  5237k      0  0:00:15  0:00:07  0:00:08 7881k
 60 80.2M   60 48.1M    0     0  5836k      0  0:00:14  0:00:08  0:00:06 9891k
 73 80.2M   73 58.8M    0     0  6379k      0  0:00:12  0:00:09  0:00:03 10.5M
 86 80.2M   86 69.5M    0     0  6815k      0  0:00

In [6]:
#Put percentVal of samples in a validation directory, to be performed only once in project
# base_dir = pathlib.Path("aclImdb")
# val_dir = base_dir / "val" 
# train_dir = base_dir / "train" 
# for category in ("neg", "pos"):
#     os.makedirs(val_dir / category)
#     files = os.listdir(train_dir / category)
#     random.Random(1337).shuffle(files)
#     num_val_samples = int(percentVal * len(files))
#     val_files = files[-num_val_samples:]
#     for fname in val_files:
#         shutil.move(train_dir / category / fname,
#                     val_dir / category / fname)

In [20]:
# Let’s create three Dataset objects for training, validation, and testing:
train_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/train", batch_size=batch_size
)
val_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/val", batch_size=batch_size
)
test_ds = keras.utils.text_dataset_from_directory(
    "aclImdb/test", batch_size=batch_size
)

Found 20000 files belonging to 2 classes.
Found 5000 files belonging to 2 classes.
Found 25000 files belonging to 2 classes.


In [21]:
train_ds = keras.utils.text_dataset_from_directory(trainPath, batch_size=batch_size)
text_only_train_ds = train_ds.map(lambda x, y: x)

text_vectorizationTfIdf = TextVectorization(ngrams=2,max_tokens=maxTokens,output_mode="tf-idf")	
text_vectorizationTfIdf.adapt(text_only_train_ds)

text_vectorizationBidiLSTM = layers.TextVectorization(max_tokens=maxTokens,output_mode="int",output_sequence_length=max_length,)
text_vectorizationBidiLSTM.adapt(text_only_train_ds)



Found 20000 files belonging to 2 classes.


In [22]:
# Displaying the shapes and dtypes of the first batch
for inputs, targets in train_ds:
    print("inputs.shape:", inputs.shape)
    print("inputs.dtype:", inputs.dtype)
    print("targets.shape:", targets.shape)
    print("targets.dtype:", targets.dtype)
    print("inputs[0]:", inputs[0])
    print("targets[0]:", targets[0])
    break


inputs.shape: (32,)
inputs.dtype: <dtype: 'string'>
targets.shape: (32,)
targets.dtype: <dtype: 'int32'>
inputs[0]: tf.Tensor(b"I've seen this movie and I must say I'm very impressed. There are not much movies I like, but I do like this one. You should see this movie by yourself and comment it,because this is one of my most favorite movie. I fancy to see this again. Action fused with a fantastic story. Very impressing. I like Modesty's character. Actually she's very mystic and mysterious (I DO like that^^). The bad boy is pretty too. Well, actually this whole movie is rare in 'movieworld'. I considered about the vote of this movie, I thought this is should be a very popular movie. I guess wrong. It was ME who was very impressed about this movie, and I hope I'm not the only one who takes only the cost to watch this one. See and vote.", shape=(), dtype=string)
targets[0]: tf.Tensor(1, shape=(), dtype=int32)


In [23]:
# Our model-building utility
  
def get_model(max_tokens=maxTokens, hidden_dim=16):
    inputs = keras.Input(shape=(max_tokens,))
    x = layers.Dense(hidden_dim, activation="relu")(inputs)
    x = layers.Dropout(0.5)(x)
    outputs = layers.Dense(1, activation="sigmoid")(x)
    model = keras.Model(inputs, outputs)
    model.compile(optimizer="rmsprop",
                  loss="binary_crossentropy",
                  metrics=["accuracy"])
    return model

In [24]:
def modelFromName(name):
    path=nlpRoot+"/models/"+name
    if name=='bagBigramTdIdf'or name=='embedBidiLSTM':
        return tf.keras.models.load_model(path) 
    elif name == "transformer":
        return tf.keras.models.load_model(path, custom_objects={"TransformerEncoder": TransformerEncoder,"PositionalEmbedding": PositionalEmbedding})
    else:
        print("Error in model name, does not exist, aborting ...")
        exit(1)
        

In [25]:
def predictSentiment(inString,modelName):
    tfString=tf.convert_to_tensor([[inString],])
    model=modelFromName(modelName)
    inputs = keras.Input(shape=(1,), dtype="string")
    if modelName=='bagBigramTdIdf':
        processed_inputs = text_vectorizationTfIdf(inputs)
    elif modelName=='embedBidiLSTM' or modelName == 'transformer':
        processed_inputs = text_vectorizationBidiLSTM(inputs)
    else:
        print("Can't vectorize this model, exiting ..")
        exit(1) 
    outputs = model(processed_inputs)
    inference_model = keras.Model(inputs, outputs)
    predictions = inference_model(tfString)
    resp=round(float(predictions[0] * 100),2)
    return resp
    

In [52]:
# Configuring the TextVectorization layer to return bigrams with TF-IDF
text_vectorizationTfIdf = TextVectorization(
    ngrams=2,
    max_tokens=maxTokens,
    output_mode="tf-idf"
)
text_vectorizationTfIdf.adapt(text_only_train_ds)

In [53]:
# Training and testing the TF-IDF bigram model
 
tfidf_2gram_train_ds = train_ds.map(
    lambda x, y: (text_vectorizationTfIdf(x), y),
    num_parallel_calls=4)
tfidf_2gram_val_ds = val_ds.map(
    lambda x, y: (text_vectorizationTfIdf(x), y),
    num_parallel_calls=4)
tfidf_2gram_test_ds = test_ds.map(
    lambda x, y: (text_vectorizationTfIdf(x), y),
    num_parallel_calls=4)
 
model = get_model()
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("tfidf_2gram.keras",
                                    save_best_only=True)
]
model.fit(tfidf_2gram_train_ds.cache(),
          validation_data=tfidf_2gram_val_ds.cache(),
          epochs=epochCount,
          callbacks=callbacks)
model = keras.models.load_model("tfidf_2gram.keras")
print(f"tfidf_2gram test accuracy: {model.evaluate(tfidf_2gram_test_ds)[1]:.3f}")

Model: "model_13"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_14 (InputLayer)       [(None, 20000)]           0         
                                                                 
 dense_15 (Dense)            (None, 16)                320016    
                                                                 
 dropout_5 (Dropout)         (None, 16)                0         
                                                                 
 dense_16 (Dense)            (None, 1)                 17        
                                                                 
Total params: 320,033
Trainable params: 320,033
Non-trainable params: 0
_________________________________________________________________
782/782 [==============================] - 65s 81ms/step - loss: 0.3252 - accuracy: 0.8709
tfidf_2gram test accuracy: 0.871


In [28]:
# Saving/loading model
path=nlpRoot+'/models'+'/bagBigramTdIdf'
model.save(path)
model=keras.models.load_model(path) # to make sure we can retrieve it later


INFO:tensorflow:Assets written to: C:/Users/lerma/machineLearning/NLP/models/bagBigramTdIdf\assets


INFO:tensorflow:Assets written to: C:/Users/lerma/machineLearning/NLP/models/bagBigramTdIdf\assets


In [29]:
print(predictSentiment(posString,'bagBigramTdIdf'),"percent positive for string",posString,'with model bagBigramTdIdf')
print(predictSentiment(negString,'bagBigramTdIdf'),"percent positive for string",negString,'with model bagBigramTdIdf')

90.49 percent positive for string That was an excellent movie, I loved it. with model bagBigramTdIdf
22.81 percent positive for string That was a horrible movie, I hated it. with model bagBigramTdIdf


In [49]:
# Now lets do embedding with bidi LSTM - Takes a while to fit, so set number of epoch to very low number
  
text_vectorizationBidiLSTM = layers.TextVectorization(
    max_tokens=maxTokens,
    output_mode="int",
    output_sequence_length=max_length
)
text_vectorizationBidiLSTM.adapt(text_only_train_ds)
 


In [ ]:
int_train_ds = train_ds.map(
    lambda x, y: (text_vectorizationBidiLSTM(x), y),
    num_parallel_calls=4)
int_val_ds = val_ds.map(
    lambda x, y: (text_vectorizationBidiLSTM(x), y),
    num_parallel_calls=4)
int_test_ds = test_ds.map(
    lambda x, y: (text_vectorizationBidiLSTM(x), y),
    num_parallel_calls=4)


inputs = keras.Input(shape=(None,), dtype="int64")
embedded = layers.Embedding(
    input_dim=maxTokens, output_dim=256, mask_zero=True)(inputs)
x = layers.Bidirectional(layers.LSTM(32))(embedded)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()
callbacks = [
    keras.callbacks.ModelCheckpoint("embeddings_bidir_gru_with_masking.keras",
                                    save_best_only=True)
]
model.fit(int_train_ds, validation_data=int_val_ds, epochs=epochCount,
          callbacks=callbacks)
model = keras.models.load_model("embeddings_bidir_gru_with_masking.keras") 
print(f"bidiLSTM test accuracy: {model.evaluate(int_test_ds)[1]:.3f}")

#Saving/loading model
path=nlpRoot+'/models'+'/embedBidiLSTM'
model.save(path)


In [31]:
print(predictSentiment(posString,'embedBidiLSTM'),"percent positive for string",posString,'with model embedBidiLSTM')
print(predictSentiment(negString,'embedBidiLSTM'),"percent positive for string",negString,'with model embedBidiLSTM')

93.12 percent positive for string That was an excellent movie, I loved it. with model embedBidiLSTM
14.72 percent positive for string That was a horrible movie, I hated it. with model embedBidiLSTM


In [32]:
# Transformer implementation
  
class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
  
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
  
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
 
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
 
    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

In [33]:
# Training and evaluation, Combining the Transformer encoder with positional embedding

inputs = keras.Input(shape=(None,), dtype="int64")
x = PositionalEmbedding(max_length, maxTokens, embed_dim)(inputs)
x = TransformerEncoder(embed_dim, dense_dim, num_heads)(x)
x = layers.GlobalMaxPooling1D()(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(1, activation="sigmoid")(x)
model = keras.Model(inputs, outputs)
model.compile(optimizer="rmsprop",
              loss="binary_crossentropy",
              metrics=["accuracy"])
model.summary()
  
callbacks = [
    keras.callbacks.ModelCheckpoint("full_transformer_encoder.keras",
                                    save_best_only=True)
] 
model.fit(int_train_ds, validation_data=int_val_ds, epochs=epochCount, callbacks=callbacks)
model = keras.models.load_model(
    "full_transformer_encoder.keras",
    custom_objects={"TransformerEncoder": TransformerEncoder,
                    "PositionalEmbedding": PositionalEmbedding}) 
print(f"For Transformer, test accuracy is: {model.evaluate(int_test_ds)[1]:.3f}")


Model: "model_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_11 (InputLayer)       [(None, None)]            0         
                                                                 
 positional_embedding (Posit  (None, None, 256)        5273600   
 ionalEmbedding)                                                 
                                                                 
 transformer_encoder (Transf  (None, None, 256)        543776    
 ormerEncoder)                                                   
                                                                 
 global_max_pooling1d (Globa  (None, 256)              0         
 lMaxPooling1D)                                                  
                                                                 
 dropout_4 (Dropout)         (None, 256)               0         
                                                          

In [34]:
#Saving model
path=nlpRoot+'/models'+'/transformer'
model.save(path,save_format='h5')


In [35]:
# print(predictSentiment(posString,'bagBigramTdIdf'),"percent positive for string",posString,'with model bagBigramTdIdf')
# print(predictSentiment(negString,'bagBigramTdIdf'),"percent positive for string",negString,'with model bagBigramTdIdf')
# print(predictSentiment(posString,'embedBidiLSTM'),"percent positive for string",posString,'with model embedBidiLSTM')
# print(predictSentiment(negString,'embedBidiLSTM'),"percent positive for string",negString,'with model embedBidiLSTM')
print(predictSentiment(posString,'transformer'),"percent positive for string",posString,'with model transformer')
print(predictSentiment(negString,'transformer'),"percent positive for string",negString,'with model transformer')

99.56 percent positive for string That was an excellent movie, I loved it. with model transformer
1.37 percent positive for string That was a horrible movie, I hated it. with model transformer


In [51]:
#save text vectorizations to file so it canbe pre-computed and loaded from streamlit app
import pickle 

fileName=nlpRoot+'/models'+'/vectorLSTM.pkl'
pickle.dump({'config': text_vectorizationBidiLSTM.get_config(),
             'weights': text_vectorizationBidiLSTM.get_weights()}
            , open(fileName, "wb"))

fileName=nlpRoot+'/models'+'/vectorTfIdf.pkl'
pickle.dump({'config': text_vectorizationTfIdf.get_config(),
             'weights': text_vectorizationTfIdf.get_weights()}
            , open(fileName, "wb"))


fileName=nlpRoot+'/models'+'/vectorLSTM.pkl'
from_disk = pickle.load(open(fileName, "rb"))
text_vectorizationBidiLSTM = TextVectorization.from_config(from_disk['config'])
# You have to call `adapt` with some dummy data (BUG in Keras)
#new_v.adapt(tf.data.Dataset.from_tensor_slices(["xyz"]))
text_vectorizationBidiLSTM.set_weights(from_disk['weights'])


In [ ]:
# streamlit application, for reference, as it's a separate python file.
import tensorflow as tf
from tensorflow.keras import layers
from tensorflow import keras
from tensorflow.keras.layers import TextVectorization
import streamlit as st
import pickle

st.title('Movie sentiment across models')


#Global variables
maxTokens = 20000 # max vocabulary size
max_length=600 # max document size
batch_size=32
# to be run at "C:/Users/plermant/git/miniprojects/NLPpractice"
nlpRoot="."

trainPath=nlpRoot+"/aclImdb/train"

@st.cache_resource
def getTrainDS(path, batch_size=batch_size):
    return keras.utils.text_dataset_from_directory(path, batch_size=batch_size)
    
@st.cache_resource 
def getTextTrainDS(_ds):
    return _ds.map(lambda x, y: x)
    
train_ds = getTrainDS(trainPath, batch_size=batch_size)
text_only_train_ds = getTextTrainDS(train_ds)

fileName=nlpRoot+'/models'+'/vectorTfIdf.pkl'
from_disk = pickle.load(open(fileName, "rb"))
text_vectorizationTfIdf = TextVectorization.from_config(from_disk['config'])
text_vectorizationTfIdf.set_weights(from_disk['weights'])


fileName=nlpRoot+'/models'+'/vectorLSTM.pkl'
from_disk = pickle.load(open(fileName, "rb"))
text_vectorizationBidiLSTM = TextVectorization.from_config(from_disk['config'])
text_vectorizationBidiLSTM.set_weights(from_disk['weights'])

class TransformerEncoder(layers.Layer):
    def __init__(self, embed_dim, dense_dim, num_heads, **kwargs):
        super().__init__(**kwargs)
        self.embed_dim = embed_dim
        self.dense_dim = dense_dim
        self.num_heads = num_heads
        self.attention = layers.MultiHeadAttention(
            num_heads=num_heads, key_dim=embed_dim)
        self.dense_proj = keras.Sequential(
            [layers.Dense(dense_dim, activation="relu"),
             layers.Dense(embed_dim),]
        )
        self.layernorm_1 = layers.LayerNormalization()
        self.layernorm_2 = layers.LayerNormalization()
    def call(self, inputs, mask=None):
        if mask is not None:
            mask = mask[:, tf.newaxis, :]
        attention_output = self.attention(
            inputs, inputs, attention_mask=mask)
        proj_input = self.layernorm_1(inputs + attention_output)
        proj_output = self.dense_proj(proj_input)
        return self.layernorm_2(proj_input + proj_output)
  
    def get_config(self):
        config = super().get_config()
        config.update({
            "embed_dim": self.embed_dim,
            "num_heads": self.num_heads,
            "dense_dim": self.dense_dim,
        })
        return config
        

class PositionalEmbedding(layers.Layer):
    def __init__(self, sequence_length, input_dim, output_dim, **kwargs):
        super().__init__(**kwargs)
        self.token_embeddings = layers.Embedding(
            input_dim=input_dim, output_dim=output_dim)
        self.position_embeddings = layers.Embedding(
            input_dim=sequence_length, output_dim=output_dim)
        self.sequence_length = sequence_length
        self.input_dim = input_dim
        self.output_dim = output_dim
  
    def call(self, inputs):
        length = tf.shape(inputs)[-1]
        positions = tf.range(start=0, limit=length, delta=1)
        embedded_tokens = self.token_embeddings(inputs)
        embedded_positions = self.position_embeddings(positions)
        return embedded_tokens + embedded_positions
 
    def compute_mask(self, inputs, mask=None):
        return tf.math.not_equal(inputs, 0)
 
    def get_config(self):
        config = super().get_config()
        config.update({
            "output_dim": self.output_dim,
            "sequence_length": self.sequence_length,
            "input_dim": self.input_dim,
        })
        return config

@st.cache_resource
def modelFromName(name):
    path=nlpRoot+"/models/"+name
    if name=='bagBigramTdIdf' or name =='embedBidiLSTM':
        return tf.keras.models.load_model(path)	
    elif name == "transformer":
        return tf.keras.models.load_model(path, custom_objects={"TransformerEncoder": TransformerEncoder,"PositionalEmbedding": PositionalEmbedding})
    else:
        print("Error in model name, does not exist, aborting ...")
        exit(1)
 
@st.cache_resource 
def predictSentiment(inString,modelName):
    tfString=tf.convert_to_tensor([[inString],])
    model=modelFromName(modelName)
    inputs = keras.Input(shape=(1,), dtype="string")
    if modelName=='bagBigramTdIdf':
        processed_inputs = text_vectorizationTfIdf(inputs)
    elif modelName=='embedBidiLSTM' or modelName == 'transformer':
        processed_inputs = text_vectorizationBidiLSTM(inputs)
    outputs = model(processed_inputs)
    inference_model = keras.Model(inputs, outputs)
    predictions = inference_model(tfString) 
    resp=round(float(predictions[0] * 100),2)
    return resp
	
modelName= st.selectbox("Select a ML model from this list",("bagBigramTdIdf", 'embedBidiLSTM',"transformer"))
inputString= st.text_input('What do you think of the movie? e.g. "I loved the movie", or "This movie sucked"')

sentimentPercent=predictSentiment(inputString,modelName)
st.write("Sentiment value is",sentimentPercent,"per",modelName,"model.")

